In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
import wave
import pyroomacoustics as pa
import glob
import random

from tqdm import tqdm

# 乱数を初期化
seed = 1
random.seed(seed)

In /home/ubuntu/miniconda3/envs/custom-env/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/ubuntu/miniconda3/envs/custom-env/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/ubuntu/miniconda3/envs/custom-env/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/ubuntu/miniconda3/envs/custom-env/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases

In [3]:
# 音声データを指定したサンプリングレートで保存
def save_audio_file(file_path, data, sample_rate):
    sf.write(file_path, data, sample_rate)

In [6]:
# 音声に室内インパルス応答（Room Impulse Response）を畳み込む
def rir_convolve(wave_files, sample_rate, audio_length, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, gain_decay, max_order=0, absorption=None, SNR=None, noise_supression=False):
    """
    wave_files: シングルチャンネルの音声のパスを格納したリスト
    sample_rate: サンプリング周波数 [Hz]
    audio_length: 音声の長さ [sec]
    doas: 音源の到来方向
    distance_mic_to_source: 音源とマイクロホンの距離 [m]
    mic_array_loc: マイクロホンアレイの位置座標
    R: 各マイクロホンの空間的な座標
    room_dim: 部屋の３次元形状を表す（単位はm）
    gain_decay: 雑音の減衰率（雑音が大きすぎるため）
    max_order: 部屋の壁で何回音が反射するか（反射しない場合0）
    absorption: 部屋の壁でどの程度音が吸収されるか （吸収されない場合None）
    SNR: 音声と雑音の比率 [dB]
    noise_supression: 雑音の音量が下げる場合True
    """
    n_sources = len(wave_files)
#     print("音源数:", n_sources)
    source_locations = np.zeros((3, doas.shape[0]), dtype=doas.dtype)
    """source_locations: (xyz, num_sources)"""
    
    source_locations[0,  :] = np.cos(doas[:, 1]) * np.cos(doas[:, 0]) # x = rcosφcosθ
    source_locations[1,  :] = np.sin(doas[:, 1]) * np.cos(doas[:, 0]) # y = rsinφcosθ
    source_locations[2,  :] = np.sin(doas[:, 0]) # z = rsinθ
    source_locations *= distance_mic_to_source
    source_locations += mic_array_loc[:, None] # マイクロホンアレイからの相対位置→絶対位置
    for i in range(n_sources):
        x = source_locations[0, i]
        y = source_locations[1, i]
        z = source_locations[2, i]
#         print("{}個目の音源の位置： (x, y, z) = ({}, {}, {})".format(i+1, x, y, z))

    # 音源数分の音声ファイルを読み込む
    for s, wave_file in enumerate(wave_files):
        audio_data, _ = sf.read(wave_file)
        """audio_data: (num_samples, )"""
        # 雑音の音量を小さくする場合
        if noise_supression:
            audio_data *= gain_decay
        if s == 0:
            clean_data = audio_data[np.newaxis, :]
        else:
            # 混合音声中の背景雑音の音量を若干小さくする（臨時処理） TODO
            if s == 2:
                audio_data *= gain_decay
            # 目的話者の発話と干渉話者の発話の長さが違う場合は目的話者の発話の長さに合わせる
            if clean_data.shape[1] < len(audio_data):
                audio_data = audio_data[:clean_data.shape[1]]
            elif clean_data.shape[1] > len(audio_data):
                audio_data = np.pad(audio_data, (0, max(0, clean_data.shape[1] - len(audio_data))), "constant")
            clean_data = np.append(clean_data, audio_data[np.newaxis, :], axis=0)
    """clean_data: (num_sources, num_samples)"""
        
    # 部屋を生成する
    room = pa.ShoeBox(room_dim, fs=sample_rate, max_order=max_order, absorption=absorption)
    # 用いるマイクロホンアレイの情報を設定する
    room.add_microphone_array(pa.MicrophoneArray(R, fs=room.fs))
    # 各音源をシミュレーションに追加する
    for s in range(n_sources):
        clean_data[s] /= np.std(clean_data[s])
        # たまに「ValueError: The source must be added inside the room.」が出る
        room.add_source(source_locations[:, s], signal=clean_data[s])
    # RIRのシミュレーション生成と音源信号への畳み込みを実行
    room.simulate(snr=SNR)
    
#     # インパルス応答の取得と残響時間（RT60）の取得
#     impulse_responses = room.rir
#     rt60 = pa.experimental.measure_rt60(impulse_responses[0][0], fs=sample_rate)
#     print("残響時間:{} [sec]".format(rt60))

    # 室内インパルス応答を畳み込んだ波形データを取得
    convolved_wave = room.mic_array.signals.T
    """convolved_wave: (num_samples, num_channels)"""
    
    return convolved_wave

In [7]:
if __name__ == '__main__':
    # 各パラメータを設定
    sample_rate = 16000 # 作成するオーディオファイルのサンプリング周波数を指定
    audio_length = 3 # 単位は秒(second) → fft_size=1024,hop_length=768のとき、audio_length=6が最適化かも？
    train_val_ratio = 0.9 # trainデータとvalidationデータの割合
    fft_size = 512 # 短時間フーリエ変換のフレーム長
    hop_length = 160 # 短時間フーリエ変換においてフレームをスライドさせる幅
    gain_decay = 0.7 # 音量調整のためのパラメータ（雑音が大きすぎるため）
    
    # RIR生成用のパラメータ
    # 畳み込みに用いる波形
    # clean_wave_files = ["../data/NoisySpeechDatabase/noisy_trainset_28spk_wav_16kHz/p230_013.wav"]
    # 音声と雑音の比率 [dB]
    SNR = None
    # 音源とマイクロホンの距離 [m]
    distance_mic_to_source=2
#     # 音源方向（音源が複数ある場合はリストに追加）
#     azimuth = [0] # 方位角
#     elevation = [np.pi/6] # 仰角
    noise_azimuth = np.pi # 雑音の方位角
    # 部屋（シミュレーション環境）の設定
    room_width = 5.0
    room_length = 5.0
    room_height = 5.0
    # 部屋の残響を設定（以下は目安）
    # max_order = 8, absorption=0.524で残響時間は0.1003750
    # max_order=22, absorption=0.333で残響時間は0.2002499
    # max_order=27, absorption=0.206で残響時間は0.3003125
    # max_order=36, absorption=0.1565で残響時間は0.4008125
    # max_order=48, absorption=0.1386で残響時間は0.5000625
    max_order = 27 # 部屋の壁で何回音が反射するか（反射しない場合0）
    absorption = 0.206 # 部屋の壁でどの程度音が吸収されるか （吸収されない場合None）
    # 以下は固定
    # 部屋の３次元形状を表す（単位はm）
    room_dim = np.r_[room_width, room_length, room_height]
    print("部屋の3次元形状：", room_dim)
    # マイクロホンアレイの中心位置
    nakbot_height = 0.57 # Nakbotの全長
    mic_array_height = nakbot_height - 0.04 # 0.04はTAMAGO-03マイクロホンアレイの頂上部からマイクロホンアレイ中心までの距離
    mic_array_loc = np.r_[room_width/2, room_length/2, 0] + [0, 0, mic_array_height] # 部屋の中央に配置されたNakbot上のマイクロホンアレイ
    print("マイクロホンアレイ中心座標：", mic_array_loc)
    # TAMAGO-03のマイクロホンアレイのマイクロホン配置（単位はm）
    mic_alignments = np.array(
    [
        [0.035, 0.0, 0.0],
        [0.035/np.sqrt(2), 0.035/np.sqrt(2), 0.0],
        [0.0, 0.035, 0.0],
        [-0.035/np.sqrt(2), 0.035/np.sqrt(2), 0.0],
        [-0.035, 0.0, 0.0],
        [-0.035/np.sqrt(2), -0.035/np.sqrt(2), 0.0],
        [0.0, -0.035, 0.0],
        [0.035/np.sqrt(2), -0.035/np.sqrt(2), 0.0]
    ])
    n_channels = np.shape(mic_alignments)[0]
    print("マイクロホン数：", n_channels)
    # get the microphone array （各マイクロホンの空間的な座標）
    R = mic_alignments.T + mic_array_loc[:, None]
    """R: (3D coordinates [m], num_microphones)"""
#     # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
#     doas = np.array(
#     [[elevation[0], azimuth[0]], # １個目の音源 
#     # [elevation[1], azimuth[1]] # ２個目の音源
#     ])
    
    # データセットを格納するディレクトリを作成
    save_dataset_dir = "../data/NoisySpeechDataset_multi_wav_test_two_speakers_rt0300/"
    os.makedirs(save_dataset_dir, exist_ok=True)
    
    # 元の長さ版
    # 目的話者の発話のディレクトリを指定
    target_data_dir_for_test= "../data/NoisySpeechDatabase/clean_testset_wav_16kHz_original_length_half1/"
    # 干渉話者の発話のディレクトリを指定
    interference_data_dir_for_test = "../data/NoisySpeechDatabase/clean_testset_wav_16kHz_original_length_half2/"
    # 外部雑音のディレクトリを指定
    noise_data_dir_for_test = "../data/NoisySpeechDatabase/interference_testset_wav_16kHz_original_length/"
    # 混合音声のディレクトリを指定
    mixed_data_dir_for_test = "../data/NoisySpeechDatabase/noisy_testset_wav_16kHz_original_length/"
    
    # テストデータのリストを作成
    # 目的話者の発話データ
    target_data_path_template_for_test = os.path.join(target_data_dir_for_test, "*.wav")
    target_list_for_test = glob.glob(target_data_path_template_for_test)
    random.seed(seed)
    target_list_for_test = random.sample(target_list_for_test, 30) # データ量削減
    # 干渉話者の発話データ
    interference_data_path_template_for_test = os.path.join(interference_data_dir_for_test, "*.wav")
    interference_list_for_test = glob.glob(interference_data_path_template_for_test)
#     random.seed(seed)
#     target_list_for_test = random.sample(target_list_for_test, 30) # データ量削減
    print("testデータの数:", len(target_list_for_test))
        
    # testデータを作成
    print("testデータ作成中")
    test_data_path = os.path.join(save_dataset_dir, "test")
    os.makedirs(test_data_path, exist_ok=True)
    
    # 干渉音の到来方向を指定（0°, 15°, 30°, 45°, 60°, 75°, 90°の7分割）
    interference_azimuth_list = [i * (np.pi / 12) for i in range(7)]
    
    for interference_azimuth_idx, interference_azimuth in enumerate(interference_azimuth_list): 
        for idx, target_path in enumerate(tqdm(target_list_for_test)):
            file_num = os.path.basename(target_path).split('.')[0] # (例)p226_001
            target_file_name = file_num + "_target.wav" # (例)p226_001_target.wav

            # 音声にRIRを畳み込みながらマルチチャンネルに拡張
            # 目的音の畳み込み
            # 音源方向（音源が複数ある場合はリストに追加、目的音の音源方向は固定）
            azimuth = [0] # 方位角（1個目の音源, 2個目の音源）
#             azimuth = [np.pi]
            elevation = [np.pi/6] # 仰角（1個目の音源, 2個目の音源）
            # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
            doas = np.array(
            [[elevation[0], azimuth[0]], # １個目の音源 
    #         [elevation[1], azimuth[1]] # ２個目の音源
            ])
            convolved_target_data = rir_convolve([target_path], sample_rate, audio_length, doas, distance_mic_to_source, \
                     mic_array_loc, R, room_dim, gain_decay, max_order=0, absorption=None, SNR=None, noise_supression=False)

            # 干渉音の畳み込み
#             interference_path = os.path.join(interference_data_dir_for_test, file_num + ".wav")    
            interference_path = random.sample(interference_list_for_test, 1)[0]
            interference_file_num = os.path.basename(interference_path).split('.')[0] # (例)p226_001
    #         # 干渉音の到来方向を指定（0°, 15°, 30°, 45°, 60°, 75°, 90°の7分割）
    #         interference_azimuth = int(idx / (len(target_list_for_test) / 7)) * (np.pi / 12)
            # 音源方向（音源が複数ある場合はリストに追加、目的音の音源方向は固定）
            azimuth = [interference_azimuth] # 方位角（1個目の音源, 2個目の音源）
            elevation = [np.pi/6] # 仰角（1個目の音源, 2個目の音源）
            # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
            doas = np.array(
            [[elevation[0], azimuth[0]], # １個目の音源 
    #         [elevation[1], azimuth[1]] # ２個目の音源
            ])
            # 音声にRIRを畳み込みながらマルチチャンネルに拡張
            convolved_interference_data = rir_convolve([interference_path], sample_rate, audio_length, doas, distance_mic_to_source, \
                     mic_array_loc, R, room_dim, gain_decay, max_order=0, absorption=None, SNR=None, noise_supression=False)

            # 雑音の畳み込み
            noise_path = os.path.join(noise_data_dir_for_test, file_num + ".wav")
            # 音源方向（音源が複数ある場合はリストに追加、目的音の音源方向は固定）
            elevation = [np.pi/6] # 仰角（1個目の音源, 2個目の音源）
            # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
            doas = np.array(
            [[elevation[0], noise_azimuth], # １個目の音源 
    #         [elevation[1], azimuth[1]] # ２個目の音源
            ])
            # 音声にRIRを畳み込みながらマルチチャンネルに拡張
            convolved_noise_data = rir_convolve([noise_path], sample_rate, audio_length, doas, distance_mic_to_source, \
                     mic_array_loc, R, room_dim, gain_decay, max_order=0, absorption=None, SNR=None, noise_supression=True)

            # 畳み込む音声をリストに格納
            wave_files = [target_path, interference_path, noise_path]
            # 音源方向（音源が複数ある場合はリストに追加、目的音の音源方向は固定）
            azimuth = [0, interference_azimuth, noise_azimuth] # 方位角（1個目の音源, 2個目の音源）
#             azimuth = [np.pi, interference_azimuth] # 20210226一時的に変更
            elevation = [np.pi/6, np.pi/6, np.pi/6] # 仰角（1個目の音源, 2個目の音源）
            # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
            doas = np.array(
            [[elevation[0], azimuth[0]], # １個目の音源 
            [elevation[1], azimuth[1]], # ２個目の音源
            [elevation[2], azimuth[2]] # 3個目の音源
            ])
            # 音声にRIRを畳み込みながらマルチチャンネルに拡張
            convolved_mixed_data = rir_convolve(wave_files, sample_rate, audio_length, doas, distance_mic_to_source, \
                     mic_array_loc, R, room_dim, gain_decay, max_order=max_order, absorption=absorption, SNR=SNR, noise_supression=False)
            # RIRの長さ-1サンプル分音声データが長くなるので、convolved_target_dataの長さに揃える
            convolved_mixed_data = convolved_mixed_data[:convolved_target_data.shape[0], :]
            """convolved_mixed_data: (num_samples, num_channels=8)"""

            # 混合音声の最大振幅で正規化
            normalized_convolved_target_data = convolved_target_data / convolved_mixed_data.max()
            normalized_convolved_interference_data = convolved_interference_data / convolved_mixed_data.max()
            normalized_convolved_noise_data = convolved_noise_data / convolved_mixed_data.max()
            normalized_convolved_mixed_data = convolved_mixed_data / convolved_mixed_data.max()
            
            # 音声データのサンプリング周波数を指定して保存
            # 干渉音の到来方向ごとに保存
#             interference_azimuth = str(int(idx / (len(target_list_for_test) / 7)) * 15) # 0, 15, 30・・・,90
            dir_name_interference_azimuth_wise = str(interference_azimuth_idx * 15)
            save_dir_azimuth_wise = os.path.join(test_data_path, dir_name_interference_azimuth_wise)
            os.makedirs(save_dir_azimuth_wise, exist_ok=True)
            # 目的音
            target_file_path = os.path.join(save_dir_azimuth_wise, target_file_name)
            save_audio_file(target_file_path, normalized_convolved_target_data, sample_rate)
            # 干渉音
            interference_file_name = file_num +"_" + interference_file_num + "_interference.wav" # (例)p226_001_p258_041_interference.wav
            interference_file_path = os.path.join(save_dir_azimuth_wise, interference_file_name)
            save_audio_file(interference_file_path, normalized_convolved_interference_data, sample_rate)
            # 雑音
            noise_file_name = file_num +"_" + interference_file_num + "_noise.wav" # (例)p226_001_p258_041_noise.wav
            noise_file_path = os.path.join(save_dir_azimuth_wise, noise_file_name)
            save_audio_file(noise_file_path, normalized_convolved_noise_data, sample_rate)
            # 混合音声
            mixed_file_name = file_num +"_" + interference_file_num + "_mixed.wav" # (例)p226_001_p258_041_mixed.wav
            mixed_file_path = os.path.join(save_dir_azimuth_wise, mixed_file_name)
            save_audio_file(mixed_file_path, normalized_convolved_mixed_data, sample_rate)

    print("データ作成完了　保存先：{}".format(save_dataset_dir))


  0%|          | 0/3500 [00:00<?, ?it/s]

部屋の3次元形状： [5. 5. 5.]
マイクロホンアレイ中心座標： [2.5  2.5  0.53]
マイクロホン数： 8
オリジナルデータの数: 5689
trainデータの数: 3500
validationデータの数: 350
testデータの数: 30
trainデータ作成中


  0%|          | 0/350 [00:00<?, ?it/s]

validationデータ作成中


  0%|          | 0/30 [00:00<?, ?it/s]

testデータ作成中


100%|██████████| 30/30 [01:48<00:00,  3.61s/it]

データ作成完了　保存先：../data/NoisySpeechDataset_multi_wav_test_original_length_two_speakers_rt0161_20210627/
